<a href="https://colab.research.google.com/github/golden77777/Deep-Learning-Exercise/blob/master/micro_community/recommend_engine_micro_community_cluster_hashtag_recomendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 「ナナコミ」のマイクロコミュニティを作成するための技術検証

現在得られるデータより，マイクロコミュニティ作成に有効な特徴量を作成することを試みる

分析の結果，「ハッシュタグ」，「投稿文の固有名詞」に着目するのが良さそう

まずはハッシュタグに対して主成分分析を行い，どの主成分がどのコミュニティの傾向に対応しているか確認する

In [23]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import gc
import matplotlib.pyplot as plt
import statistics
import seaborn as sns
import ast
import sklearn #機械学習のライブラリ
from sklearn.decomposition import PCA #主成分分析器
%matplotlib inline
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
# URL によるリソースへのアクセスを提供するライブラリをインポートする。
import urllib.request
from sklearn.metrics.pairwise import pairwise_distances
from surprise import Dataset, Reader, accuracy
from surprise.model_selection import train_test_split
from surprise import (
    NormalPredictor, BaselineOnly, KNNBasic, KNNWithMeans, KNNWithZScore,
    KNNBaseline, SVD, SVDpp, NMF, SlopeOne, CoClustering
)


# サービスアカウントのキーを通す（API接続のため）
import os
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/content/gaudiy-develop-tokyo-ed2268d90d1a.json'

PARAMS = {
    'target_tenant': 'nananiji-qicpv',
    'activity_measure': 'like', #pv, like, reply
    'activity_measure_nunc': 'pv', #pv, like, reply
    'excludes_official_contents': True,
    'n_contents_to_recommend': 10,
    'n_related_users': 10,
}



In [24]:
import numpy
import pandas
import random

from matplotlib import pyplot as plt
from scipy.spatial import distance
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import KMeans

# 従来の特徴量・クラスタに対してユーザレコメンドエンジンを適用することを試みる

In [121]:
df_sales = pd.read_csv("nanacomi_micro_community_new_227_new_hasutag_new_0805_cluster_manytag_text.csv", index_col = ["user_id"])

## クラスタ７に対するユーザレコメンドの結果

In [125]:
#df_piv.columns[df_sales.columns != 'user_id']
df_sales_clus7 = df_sales[df_sales["cluster_3"] == 7]

In [127]:
df_sales_clus7

79

In [128]:
len(df_sales_clus7)

79

In [136]:
df_sales_clus7_analy.head(10)

,like_number_log_norm,pv_number_log_norm,sentence_length_log_norm,hasutag_22_7_sum,hasutag_Q_A _sum,hasutag_あなたと六番町学院_sum,hasutag_ちはるんに感謝を伝えよう_sum,hasutag_はじめまして_sum,hasutag_れいにゃんに感謝を伝えよう_sum,hasutag_アニラ2022_sum,hasutag_イベント企画・告知_sum,hasutag_サポート広場,hasutag_ショートストーリー朗読劇,hasutag_チャットルームで話そう,hasutag_ナナオン向上委員会,hasutag_ナナニジキャラライブ,hasutag_ナナニジライブ2023,hasutag_ファンアート,hasutag_フレンド募集,hasutag_一之瀬蛍,hasutag_丸山あかね,hasutag_僕は今夜、出ていく,hasutag_八神叶愛,hasutag_創作SS,hasutag_四条月,hasutag_天城サリー,hasutag_宮瀬玲奈,hasutag_後輩ちゃん,hasutag_攻略情報,hasutag_望月りの,hasutag_桐生塔子,hasutag_椎名桜月,hasutag_気の抜けたサイダー,hasutag_永峰楓,hasutag_河瀬詩,hasutag_涼花萌,hasutag_滝川みう,hasutag_白沢かなえ,hasutag_神木みかみ,hasutag_神様だって決められない,hasutag_立川絢香,hasutag_聖地巡礼,hasutag_藤間桜,hasutag_西條和,hasutag_西浦そら,hasutag_22/7,hasutag_Q&A,hasutag_あなたと六番町学院,hasutag_ちはるんに感謝を伝えよう。,hasutag_はじめまして,hasutag_れいにゃんに感謝を伝えよう,hasutag_アニラ2022,hasutag_イベント企画・告知
user_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1cbZLbqD8ZagS8QW3GoI5Vb3AUK2,-3.909730,-6.534084,-3.059285,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
fCdj4MALY0bjXmQNLK95OnScp322,3.346515,3.378400,-7.452754,0,0,0,1,0,0,0,0,0,2,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
3B4SXS1skEWpO4h6JJE4Qd8AmHj2,1.928106,-6.041195,-9.631928,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0
xPpPxEjrsoNdNEFLUsbQdp69ta03,-1.794129,-0.038170,-0.658787,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
ypFGXYRdQMZYgbmHTqXqyaXibz13,-26.700744,12.120842,-3.634708,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
77AVpaz3INURSR9YVRTESmbCMfJ3,-0.816331,-0.929827,-1.746368,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
BcfMXWNJupV8FgEKCl60LqsCYyF3,-1.501653,-2.608326,-3.881853,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
qM2Zwf7MGYhp6dYsAAlbwsX3Sts2,-2.973562,-0.538536,-2.168490,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
O3AQPXQ3cyVrs5Wh46hzF02v0uZ2,-4.318640,-6.772649,-7.185307,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0


In [133]:
df_sales_clus7_analy = df_sales_clus7.iloc[:,8:61]

In [134]:
User = "3B4SXS1skEWpO4h6JJE4Qd8AmHj2"

In [135]:
df_sp = csr_matrix(df_sales_clus7.iloc[:,8:61].values)
#KNNのインスタンス化。評価方法をコサイン類似度に設定。bruteは総当り方式。
rec = NearestNeighbors(n_neighbors=5,algorithm= "brute", metric= "cosine")
# KNNで訓練
rec_model = rec.fit(df_sp)
#User = "3B4SXS1skEWpO4h6JJE4Qd8AmHj2"#ユーザーIDに似ているユーザーを見つける。
distance, indice = rec_model.kneighbors(df_sales_clus7.iloc[:,8:61].iloc[df_sales_clus7.iloc[:,8:61].index== User].values.reshape(1,-1),n_neighbors=5)
print(distance)
print(indice)
for i in range(5):#i=0はユーザーID100のことであり、i=1は最も似ているユーザーのこと。
    if  i == 0:#ユーザーID100のこと。自分自身。
        print("ユーザーID:{0}に似たユーザーID)は以下になります。".format(User))
    else:
        print('ユーザーID:{0} '.format(df_sales.iloc[:,8:62].index[indice.flatten()[i]]))
        simmilaruser = df_sales.iloc[:,8:62].index[indice.flatten()[i]]
        # rec_movie = df_sales.iloc[:,4:62][["user_id"] == df_sales.iloc[:,4:62].index[indice.flatten()[i]]]
        # rec_movie_sort = sorted(rec_movie.items(), reverse=True, key=lambda x:x[1])#reverse→降順、辞書のvalueをkeyにソート
        # rec_movie_3 = rec_movie_sort[0:3]#評価の高い映画を3つ抽出
        #print(simmilaruser)

[[0.         0.06740931 0.07101062 0.07210081 0.08358099]]
[[ 2 49 51 23 59]]
ユーザーID:3B4SXS1skEWpO4h6JJE4Qd8AmHj2に似たユーザーID)は以下になります。
ユーザーID:CkWW6HXoY9cJPchqMprNvQR11Ej1 
ユーザーID:3B4SXS1skEWpO4h6JJE4Qd8AmHj2 
ユーザーID:fCdj4MALY0bjXmQNLK95OnScp322 
ユーザーID:3XQmaEkoZbefIqHF8eRm2ggdeq42 


In [ ]:
class BaseModel:
    def __init__(self):
        self.name = "BaseModel"
        self.master_data = None
        self.similarities = None

    def train(self, train_data):
        train_data.fillna(0, inplace=True)
        self.master_data = train_data
        sims = cosine_similarity(train_data.transpose())
        item_ids = train_data.columns
        self.similarities = {
            item_id: sim for item_id, sim in zip(
                item_ids, sims)}
        return self

    def recommend(self, target_user_history):
        if self.similarities is None:
            return "BaseModel not trained."

        if not target_user_history:
            random_id = random.choice(self.master_data.columns)
            return random_id

        score_averaged = numpy.array(
            [score for _, score in target_user_history]).mean()
        scored_ids = []
        tmp_df = pandas.DataFrame(columns=self.master_data.columns)
        for scored_item_id, score in target_user_history:
            tmp_df[scored_item_id] = [score]
            scored_ids.append(scored_item_id)
        scores = tmp_df.fillna(0).values

        pred_scores = {}
        for item_id in self.master_data.columns:
            if item_id not in scored_ids:
                pred_scores[item_id] = (
                    self.similarities[item_id] * scores).sum() / self.similarities[item_id].sum()

        recommend_item_id = max(pred_scores, key=pred_scores.get)
        return recommend_item_id


class ClusteringModel:
    def __init__(self):
        self.name = "ClusteringModel"
        self.master_data = None
        self.similarities = None
        self.item_cluster_centers = None
        self.user_cluster_centers = None
        self.kmeans_model = KMeans(
            n_clusters=5,
            init="k-means++")

    def train(self, train_data):
        train_data.fillna(0, inplace=True)
        self.master_data = train_data
        sims = cosine_similarity(train_data.transpose())
        item_ids = train_data.columns
        self.similarities = {
            item_id: sim for item_id, sim in zip(
                item_ids, sims)}
        self.kmeans_model.fit(train_data)
        self.user_cluster_centers = self.kmeans_model.cluster_centers_
        self.master_data["label"] = self.kmeans_model.labels_
        self.kmeans_model.fit(train_data.transpose())
        self.item_cluster_centers = self.kmeans_model.cluster_centers_
        self.master_data.loc["label"] = self.kmeans_model.labels_ + [100]
        return self

    def recommend(self, target_user_history):
        if self.similarities is None and self.cluster_ids is None:
            return "ClusteringModel not trained."

        if not target_user_history:
            random_id = random.choice(self.master_data.columns)
            return random_id

        score_averaged = numpy.array(
            [score for _, score in target_user_history]).mean()
        scored_ids = []
        tmp_df = pandas.DataFrame(columns=self.master_data.columns)
        for scored_item_id, score in target_user_history:
            tmp_df[scored_item_id] = [score]
            scored_ids.append(scored_item_id)
        scores = tmp_df.fillna(0).values

        for center in self.user_cluster_centers:
            d = self._distance(tmp_df.fillna(0), center)
            distance_list.append(pandas.Series(d))
        df_distances = pandas.concat(distance_list, axis=1)
        user_cluster_id = df_distances.idxmin(axis=1)
        same_user_data = self.master_data[self.master_data["label"]== user_cluster_id]

        pred_scores = {}
        for item_id in self.master_data.columns:
          if item_id not in scored_ids:
            pred_scores[item_id] = ((self.similarities[item_id] *(scores - score_averaged)).sum() /self.similarities[item_id].sum()) + (((same_user_data - same_user_data.mean()).sum())/ len(same_user_data))
            print("チャント実行していてえらい")
        print(pred_scores)
        recommend_item_id = max(pred_scores, key=pred_scores.get)
        return recommend_item_id

    def _distance(self, df, centers):
        def _d(x):
            return distance.euclidean(x.values, centers)
        dist = df.apply(_d, axis=1)
        return dist.values


# def main():
#     data = numpy.loadtxt("u.data",
#                          delimiter="\t",
#                          usecols=(0, 1, 2),
#                          skiprows=1)

#     user_ids = list(set([row[0] for row in data]))
#     movie_ids = list(set([row[1] for row in data]))
#     df = pandas.DataFrame(index=user_ids, columns=movie_ids)
#     for row in data:
#         df[row[1]].loc[row[0]] = row[2]

#     train_data = df.loc[101:]
#     test_data = df.loc[:100]

#     models = [BaseModel(), ClusteringModel()]
#     for model in models:
#         model.train(train_data)
#         history_list = []
#         history = []

#         for _, row in test_data.iterrows():
#             itr = 0
#             while itr < 20:
#                 id_ = model.recommend(history)
#                 history.append((id_, row[id_]))
#                 if not numpy.isnan(row[id_]):
#                     itr += 1
#             history_list.append(history)
#         print("hoge")
#         plot_figures(history_list, model.name)


# def plot_figures(history_list, model_name):
#     fig = plt.figure()
#     ax = fig.add_subplot(1, 1, 1)
#     x = numpy.linspace(0, 100, recommend_iteration)
#     user_scores = []
#     for history in log_list:
#         scores_tmp = [score for _, score in history]
#         scores = []
#         for i, _ in enumerate(scores_tmp):
#             scores.append(max(scores_tmp[0:i + 1]))
#         user_scores.append(scores)
#     y = numpy.array(user_scores).mean(axis=0)
#     ax.plot(x, y, linestyle='solid', linewidth=3.0)
#     ax.set_xlabel('n_iteration')
#     ax.set_ylabel('score')
#     ax.set_xlim(0, 100)
#     ax.set_ylim(0, 1)
#     ax.legend()
#     ax.grid(True)
#     fig.savefig("{}.png".format(model_name))


# if __name__ == "__main__":
#   main()

In [ ]:
  data = numpy.loadtxt("u.data",
                        delimiter="\t",
                        usecols=(0, 1, 2),
                        skiprows=1)

  user_ids = list(set([row[0] for row in data]))
  movie_ids = list(set([row[1] for row in data]))
  df = pandas.DataFrame(index=user_ids, columns=movie_ids)
  for row in data:
      df[row[1]].loc[row[0]] = row[2]

  train_data = df.loc[101:]
  test_data = df.loc[:100]

  models = [BaseModel(), ClusteringModel()]
  for model in models:
      model.train(train_data)
      history_list = []
      history = []

      for _, row in test_data.iterrows():
          itr = 0
          while itr < 20:
              id_ = model.recommend(history)
              history.append((id_, row[id_]))
              if not numpy.isnan(row[id_]):
                  itr += 1
          history_list.append(history)
      print("hoge")
      plot_figures(history_list, model.name)

hoge


/usr/local/lib/python3.8/dist-packages/pandas/core/frame.py:5176: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(
<ipython-input-20-d932a63cc04d>:37: RuntimeWarning: invalid value encountered in double_scalars
  pred_scores[item_id] = (


ValueError: ignored

In [25]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [26]:
df_new_micro_hashtag = pd.read_csv("nanacomi_micro_community_new_227_new_hasutag_new_0805_cluster_manytag_text.csv")

In [ ]:
df_new_micro_hashtag_all = pd.read_csv("nanacomi_micro_community_new_227_new_hasutag_new_0731_cluster_manytag.csv")

In [138]:
#必要なモジュールをインポート
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
#pandasライブラリで読み込み。自動的にカラム名設定。
df = pd.read_csv("ratings.csv", sep=",")
#最初の0~2列目までを利用（ユーザーID、映画ID、評価の3列を利用）
df = df.iloc[:,0:3]
#疎行列(行列の中身が0ばかりの行列)に変換。行名が映画のID、列名がユーザーID、行列の中身が評価
df_piv = df.pivot(index= "movieId",columns="userId",values="rating").fillna(0)
#scikit-learnでの処理が速くなるデータ形式に変換
df_sp = csr_matrix(df_piv.values)
#KNNのインスタンス化。評価方法をコサイン類似度に設定。bruteは総当り方式。
rec = NearestNeighbors(n_neighbors=10,algorithm= "brute", metric= "cosine")
# KNNで訓練
rec_model = rec.fit(df_sp)
# 映画ID「100」に対するおすすめの映画10個を類似度(下記のdistances)が近い順番に表示。
#下記のindicesは各点から近い順にインデックスが入っており、行iには点iに近い順にインデックスが入っている。
# 下記のn_neighbors=11は自身を含めて11個のデータを取り出す。
Movie = 100#任意の好きな映画ID
distance, indice = rec_model.kneighbors(df_piv.iloc[df_piv.index== Movie].values.reshape(1,-1),n_neighbors=11)
for i in range(11):#i=0は映画ID100のことであり、i=1~10までは評価が似ている映画のこと。
    if  i == 0:#i=0はID100のこと。
        print("映画ID{}を見た人におすすめの映画IDは以下です。".format(Movie))
    else:
        print("{0}: {1}".format(i,df_piv.index[indice.flatten()[i]]))#似ている順に映画IDを表示。

映画ID100を見た人におすすめの映画IDは以下です。
1: 26
2: 354
3: 1006
4: 61
5: 667
6: 74
7: 981
8: 1082
9: 881
10: 76


In [33]:
#必要なモジュールをインポート
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
#pandasライブラリで読み込み。自動的にカラム名設定。
df = pd.read_csv("ratings.csv", sep=",")
#最初の0~2列目までを利用（ユーザーID、映画ID、評価の3列を利用）
df = df.iloc[:,0:3]
#疎行列(行列の中身が0ばかりの行列)に変換。行名がユーザーID、列名が映画のID、行列の中身が評価
df_piv = df.pivot(index= "userId",columns="movieId",values="rating").fillna(0)
#scikit-learnでの処理が速くなるデータ形式に変換
df_sp = csr_matrix(df_piv.values)
#KNNのインスタンス化。評価方法をコサイン類似度に設定。bruteは総当り方式。
rec = NearestNeighbors(n_neighbors=10,algorithm= "brute", metric= "cosine")
# KNNで訓練
rec_model = rec.fit(df_sp)
User = 100#ユーザーID100に似ているユーザーを見つける。
distance, indice = rec_model.kneighbors(df_piv.iloc[df_piv.index== User].values.reshape(1,-1),n_neighbors=2)
for i in range(2):#i=0はユーザーID100のことであり、i=1は最も似ているユーザーのこと。
    if  i == 0:#ユーザーID100のこと。自分自身。
        print("ユーザーID:{0}に似たユーザーID,(おすすめ映画ID,評価)は以下になります。".format(User))
    else:
        print('ユーザーID:{0} '.format(df_piv.index[indice.flatten() [i]]))
        rec_movie = df_piv[df_piv.index[indice.flatten() [i]]]
        rec_movie_sort = sorted(rec_movie.items(), reverse=True, key=lambda x:x[1])#reverse→降順、辞書のvalueをkeyにソート
        rec_movie_3 = rec_movie_sort[0:3]#評価の高い映画を3つ抽出
        print(rec_movie_3)

ユーザーID:100に似たユーザーID,(おすすめ映画ID,評価)は以下になります。
ユーザーID:357 
[(8, 5.0), (38, 5.0), (66, 5.0)]


In [46]:
df_piv.columns[df_piv.columns != 'user_id']

Index(['like_number', 'reply_number', 'pv_number', 'word_length',
       'like_number_log_norm', 'pv_number_log_norm',
       'sentence_length_log_norm', 'hasutag_22_7_sum', 'hasutag_Q_A _sum',
       'hasutag_あなたと六番町学院_sum', 'hasutag_ちはるんに感謝を伝えよう_sum',
       'hasutag_はじめまして_sum', 'hasutag_れいにゃんに感謝を伝えよう_sum',
       'hasutag_アニラ2022_sum', 'hasutag_イベント企画・告知_sum', 'hasutag_サポート広場',
       'hasutag_ショートストーリー朗読劇', 'hasutag_チャットルームで話そう', 'hasutag_ナナオン向上委員会',
       'hasutag_ナナニジキャラライブ', 'hasutag_ナナニジライブ2023', 'hasutag_ファンアート',
       'hasutag_フレンド募集', 'hasutag_一之瀬蛍', 'hasutag_丸山あかね', 'hasutag_僕は今夜、出ていく',
       'hasutag_八神叶愛', 'hasutag_創作SS', 'hasutag_四条月', 'hasutag_天城サリー',
       'hasutag_宮瀬玲奈', 'hasutag_後輩ちゃん', 'hasutag_攻略情報', 'hasutag_望月りの',
       'hasutag_桐生塔子', 'hasutag_椎名桜月', 'hasutag_気の抜けたサイダー', 'hasutag_永峰楓',
       'hasutag_河瀬詩', 'hasutag_涼花萌', 'hasutag_滝川みう', 'hasutag_白沢かなえ',
       'hasutag_神木みかみ', 'hasutag_神様だって決められない', 'hasutag_立川絢香', 'hasutag_聖地巡礼',
       'hasutag_藤間桜', 'h

In [32]:
df_piv.head()

userId,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511,512,513,514,515,516,517,518,519,520,521,522,523,524,525,526,527,528,529,530,531,532,533,534,535,536,537,538,539,540,541,542,543,544,545,546,547,548,549,550,551,552,553,554,555,556,557,558,559,560,561,562,563,564,565,566,567,568,569,570,571,572,573,574,575,576,577,578,579,580,581,582,583,584,585,586,587,588,589,590,591,592,593,594,595,596,597,598,599,600,601,602,603,604,605,606,607,608,609,610
movieId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,4.0,0.0,0.0,0.0,4.0,0.0,4.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.5,0.0,4.5,3.5,4.0,0.0,3.5,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,5.0,3.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,5.0,3.0,4.0,5.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,3.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,5.0,4.0,0.0,4.0,0.0,2.5,0.0,0.0,5.0,0.0,4.5,0.0,0.0,0.5,0.0,4.0,0.0,0.0,0.0,2.5,0.0,0.0,0.0,4.0,0.0,0.0,3.0,3.0,4.0,0.0,3.0,0.0,0.0,5.0,0.0,4.5,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,3.5,0.0,4.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,2.0,0.0,3.0,4.0,0.0,4.0,0.0,0.0,3.0,4.0,0.0,0.0,3.5,5.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,2.0,0.0,3.0,4.0,0.0,0.0,4.5,4.0,4.0,0.0,0.0,0.0,0.0,5.0,3.5,0.0,4.5,0.0,5.0,0.0,0.0,0.0,0.0,0.0,5.0,4.0,4.0,0.0,0.0,4.0,0.0,0.0,4.0,4.0,0.0,0.0,0.0,0.0,4.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,3.5,5.0,4.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,3.5,3.0,0.0,3.0,4.0,0.0,3.5,5.0,0.0,0.0,3.5,0.0,0.0,3.5,0.0,0.0,5.0,0.0,0.0,3.5,3.0,5.0,0.0,0.0,0.0,0.0,4.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,4.0,0.0,0.0,4.5,0.0,4.5,0.0,0.0,0.0,0.0,0.0,0.0,0

In [50]:
df_origi = df_new_micro_hashtag.iloc[:,4:62].head(3)

In [43]:
df_piv = df_new_micro_hashtag.iloc[:,4:62].head(3)

In [54]:
df_sales = pd.read_csv("nanacomi_micro_community_new_227_new_hasutag_new_0805_cluster_manytag_text.csv", index_col = ["user_id"])

In [137]:
df_piv.columns[df_piv.columns != 'user_id']
df_sales[df_sales["cluster_3"] == 20].head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,0,like_number,reply_number,pv_number,word_length,like_number_log_norm,pv_number_log_norm,sentence_length_log_norm,hasutag_22_7_sum,hasutag_Q_A _sum,hasutag_あなたと六番町学院_sum,hasutag_ちはるんに感謝を伝えよう_sum,hasutag_はじめまして_sum,hasutag_れいにゃんに感謝を伝えよう_sum,hasutag_アニラ2022_sum,hasutag_イベント企画・告知_sum,hasutag_サポート広場,hasutag_ショートストーリー朗読劇,hasutag_チャットルームで話そう,hasutag_ナナオン向上委員会,hasutag_ナナニジキャラライブ,hasutag_ナナニジライブ2023,hasutag_ファンアート,hasutag_フレンド募集,hasutag_一之瀬蛍,hasutag_丸山あかね,hasutag_僕は今夜、出ていく,hasutag_八神叶愛,hasutag_創作SS,hasutag_四条月,hasutag_天城サリー,hasutag_宮瀬玲奈,hasutag_後輩ちゃん,hasutag_攻略情報,hasutag_望月りの,hasutag_桐生塔子,hasutag_椎名桜月,hasutag_気の抜けたサイダー,hasutag_永峰楓,hasutag_河瀬詩,hasutag_涼花萌,hasutag_滝川みう,hasutag_白沢かなえ,hasutag_神木みかみ,hasutag_神様だって決められない,hasutag_立川絢香,hasutag_聖地巡礼,hasutag_藤間桜,hasutag_西條和,hasutag_西浦そら,hasutag_22/7,hasutag_Q&A,hasutag_あなたと六番町学院,hasutag_ちはるんに感謝を伝えよう。,hasutag_はじめまして,hasutag_れいにゃんに感謝を伝えよう,hasutag_アニラ2022,hasutag_イベント企画・告知,index,cluster_1,cluster_2,cluster_3,0.1,user_description_all
user_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
5Zfydlz0ShcETFB50dM4a7BoQLp2,5,68,68,493,87,1,11,110,0.013555,-2.808209,-1.936077,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,68,14,1,20,317,謎の語感イヤモニを付けてたり今までに無い演出があったりと環境・演出面も良かったし、メンバーの...
tBgXU9ev8Sesw7t5UpfSVAjEFCq1,11,103,103,155,185,6,44,188,-2.435693,-5.845899,-2.427443,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,103,14,1,20,832,イラストを練習中ので，何の型式のケーキを考えている。\n皆さんは提案ありますか？\n想像中の...
66m4DiOO4UQW6ux2XeFQQvKzmth2,16,146,146,1350,141,2,24,182,-0.980259,-2.807718,-0.802028,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,146,14,1,20,1380,音ゲーはじめて間もないから難しいけど少しずつ頑張ってる。そういえば育成のコツとかあるのかな？...
DdgHisxDzpPtGLa9RKNqoKrCWG42,46,301,301,391,190,24,350,921,-7.535407,0.658132,-2.850202,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,301,14,1,20,1185,頑張って上達を目指してるんですが、センチメンタルのここを突破することが出来ないままイベント後...
5k9CtgROvNZasYIAO4FwZE30Et63,47,305,305,94,330,6,35,297,1.261638,-9.005543,-4.396847,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,305,14,1,20,1160,今日初めてEXイラストをゲットしましたが、とても嬉しいものですね。\nそこで、スクショで保存...


In [55]:
df_sales.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,0,like_number,reply_number,pv_number,word_length,like_number_log_norm,pv_number_log_norm,sentence_length_log_norm,hasutag_22_7_sum,hasutag_Q_A _sum,hasutag_あなたと六番町学院_sum,hasutag_ちはるんに感謝を伝えよう_sum,hasutag_はじめまして_sum,hasutag_れいにゃんに感謝を伝えよう_sum,hasutag_アニラ2022_sum,hasutag_イベント企画・告知_sum,hasutag_サポート広場,hasutag_ショートストーリー朗読劇,hasutag_チャットルームで話そう,hasutag_ナナオン向上委員会,hasutag_ナナニジキャラライブ,hasutag_ナナニジライブ2023,hasutag_ファンアート,hasutag_フレンド募集,hasutag_一之瀬蛍,hasutag_丸山あかね,hasutag_僕は今夜、出ていく,hasutag_八神叶愛,hasutag_創作SS,hasutag_四条月,hasutag_天城サリー,hasutag_宮瀬玲奈,hasutag_後輩ちゃん,hasutag_攻略情報,hasutag_望月りの,hasutag_桐生塔子,hasutag_椎名桜月,hasutag_気の抜けたサイダー,hasutag_永峰楓,hasutag_河瀬詩,hasutag_涼花萌,hasutag_滝川みう,hasutag_白沢かなえ,hasutag_神木みかみ,hasutag_神様だって決められない,hasutag_立川絢香,hasutag_聖地巡礼,hasutag_藤間桜,hasutag_西條和,hasutag_西浦そら,hasutag_22/7,hasutag_Q&A,hasutag_あなたと六番町学院,hasutag_ちはるんに感謝を伝えよう。,hasutag_はじめまして,hasutag_れいにゃんに感謝を伝えよう,hasutag_アニラ2022,hasutag_イベント企画・告知,index,cluster_1,cluster_2,cluster_3,0.1,user_description_all
user_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
IdHAE80h0HhKWVr27FFZkzmn4Kr1,0,9,9,1178,8,3,90,133,-3.996115,-0.747289,-0.879568,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,9,9,1,58,473,隠すとこ隠せていればよくね？（柊つぼみ）■内容\n「22/7 計算中」2巻の限定シリアルコー...
1Wyr0bfamEem45qbBWEnyX6tVcJ2,1,32,32,745,444,5,126,494,-2.738953,-4.342100,-2.950358,0,0,0,0,0,0,1,0,0,0,0,0,0,0,2,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,32,3,1,29,478,星４にサインがつくと、とっても嬉しい！メンバー達の理想のシチュエーションで目覚ましをボイスド...
W1o9IKq3jLcHu6NOct3e7jy17bv1,2,34,34,1104,1614,18,915,2668,-18.339297,-27.020361,-22.329211,1,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1,0,34,14,11,38,1188,ナナオン楽しみましょうね🎶ダンスしてる時凄く輝いてるまさにアイドル興味深そうに耳引っ張ってた...
MdqIJXsNe5OJaHl6mj89xC5Wecn2,3,54,54,762,412,102,174,2099,-122.830746,-177.557470,-57.943341,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,54,6,21,4,1279,Twitter@GUMI7896\nよろしくお願いします！ほんとに開発者に怒りながら報告して...
TYfoosXNlTP7ePsELCUEFidrXno1,4,66,66,204,584,49,663,623,-12.400893,0.137612,-22.516652,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,66,6,11,38,1490,箱推しです。\n衣装、レッスン着にハマる今日この頃。なんとか終わりました！1000位以内を、...
